In [ ]:
# Load libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import re
# plotly
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
## Data Cleaning 

# Loading the response dataset from Kaggle Survey
kaggle_multiple_choice = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv', low_memory=False)

# Separating questions from answers
# This Series stores all questions
kaggle_questions = kaggle_multiple_choice.iloc[0,:]

# This DataFrame stores all answers
kaggle = kaggle_multiple_choice.iloc[1:,:]

In [ ]:
# removing everyone that took less than 4 minutes or more than 180 minutes to answer the survey (altogether 1%)

answers_before = kaggle.shape[0]
print(f'Initial dataset length is {answers_before} answers.')

# Identify those who took less than 4 min
less_4_minutes = kaggle[round(kaggle.iloc[:,0].astype(int) / 60) <= 4].index

# Dropping those rows
kaggle = kaggle.drop(less_4_minutes, axis=0)

# Identify those who took more than 180 min
more_180_minutes = kaggle[round(kaggle.iloc[:,0].astype(int) / 60) >= 180].index
kaggle = kaggle.drop(more_180_minutes, axis=0)

answers_after = kaggle.shape[0]
print('After removing respondents that took less than 4 minutes or more than 180 minutes' \
      f'to answer the survey we were left with {answers_after} answers.')

# removing respondents who are neither man nor woman
answers_before = kaggle.shape[0]

non_woman_man = kaggle[(kaggle.Q2 == 'Prefer not to say') | \
                             (kaggle.Q2 == 'Prefer to self-describe') | \
                             (kaggle.Q2 == 'Nonbinary')
                            ].index
# Dropping rows
kaggle = kaggle.drop(list(non_woman_man), axis=0)
answers_after = kaggle.shape[0]
print(f'After removing respondents who are not man or woman we were left with {answers_after} answers.')

In [ ]:
# Simplifying age groups
kaggle.Q1 = kaggle.Q1.replace('18-21', '18-24')
kaggle.Q1 = kaggle.Q1.replace('22-24', '18-24')
kaggle.Q1 = kaggle.Q1.replace('25-29', '25-29')
kaggle.Q1 = kaggle.Q1.replace('30-34', '30-39')
kaggle.Q1 = kaggle.Q1.replace('35-39', '30-39')
kaggle.Q1 = kaggle.Q1.replace('40-44', '40-49')
kaggle.Q1 = kaggle.Q1.replace('45-49', '40-49')
kaggle.Q1 = kaggle.Q1.replace('50-54', '50-59')
kaggle.Q1 = kaggle.Q1.replace('55-59', '50-59')
kaggle.Q1 = kaggle.Q1.replace('60-69', '60+')
kaggle.Q1 = kaggle.Q1.replace('70+', '60+')

In [ ]:
# add a new column 'Employed' transformed from 'Q5' (Job Titles)
def employed(row):
    if row == 'NaN':
        return 'Not answered'
    elif row == 'Currently not employed':
        return 'Unemployed'
    else:
        return 'Employed'

kaggle['Employed'] = kaggle['Q5'].apply(lambda row: employed(row))

In [ ]:
# Now lets rename some columns to have more meaningfull names
kaggle.columns = kaggle.columns.str.replace('Q20', 'CompanySize')
kaggle.columns = kaggle.columns.str.replace('Q24', 'Compensation')
kaggle.columns = kaggle.columns.str.replace('Q15', 'MLExp')
kaggle.columns = kaggle.columns.str.replace('Q1', 'Age')
kaggle.columns = kaggle.columns.str.replace('Q2', 'Gender')
kaggle.columns = kaggle.columns.str.replace('Q5', 'Title')
kaggle.columns = kaggle.columns.str.replace('Q3', 'Country')
kaggle.columns = kaggle.columns.str.replace('Q6', 'CodeExp')
kaggle.columns = kaggle.columns.str.replace('Q4', 'HighestEdu')


In [ ]:
# To create good plots, we need to transform some columns into categories.
# This is because a category might have a logical order, that will be preserved when plotting.
# Otherwise the categories would be sorted by alphabetical order

# Transform CodeExperience column into category
time_writting_code = ['I have never written code', '< 1 years', '1-2 years', '3-5 years',
                      '5-10 years', '10-20 years', '20+ years']
cat_dtype = pd.api.types.CategoricalDtype(categories=time_writting_code, ordered=True)
kaggle.CodeExp = kaggle.CodeExp.astype(cat_dtype)
# Now CodeExperience has a specific order as defined by the list time_writting_code

# We will do the same for CompanySize and transform the column into category
company_size = ['0-49 employees', '50-249 employees', '250-999 employees', '1000-9,999 employees', '10,000 or more employees']
cat_dtype = pd.api.types.CategoricalDtype(categories=company_size, ordered=True)
kaggle.CompanySize = kaggle.CompanySize.astype(cat_dtype)

# We will do the same for Age and transform the column into category
age = ['18-24', '25-29', '30-39', '40-49', '50-59','60+']
cat_dtype = pd.api.types.CategoricalDtype(categories=age, ordered=True)
kaggle.Age = kaggle.Age.astype(cat_dtype)

In [ ]:
# Add count column to make groupby easier
kaggle['Count'] = 1

In [ ]:
# Filtering only the columns we will need
kaggle = kaggle[['Age', 'Gender','HighestEdu','Country', 'Title', 'Employed','CompanySize', 'Compensation', 'CodeExp',
                 'MLExp','Count']]

# Finally our Dataframe looks like this 
kaggle.head(5)

Finding 1:

The unemployed rate in woman participants surpassed man since the age group 25-29

In [ ]:
## Data analysis
# % of Unemployed survey participants in gender
kaggle_employed = kaggle[['Gender','Age','Employed','Count']]
kaggle_employed = kaggle_employed.groupby(['Gender','Age','Employed'])['Count'].sum().reset_index()
kaggle_unemployed  = kaggle_employed.loc[kaggle_employed['Employed'] == "Unemployed"]
kaggle_emp_total = kaggle_employed.groupby(['Gender','Age'])['Count'].sum().reset_index()
kaggle_emp_total.columns = kaggle_emp_total.columns.str.replace('Count', 'total')
kaggle_employ_share = pd.merge(
    kaggle_unemployed,
    kaggle_emp_total,
    how="inner",
    on=['Gender','Age'],
    sort=True,
    copy=True,
    indicator=False,
    validate=None
)

kaggle_employ_share['Unemployed_share'] = round(kaggle_employ_share['Count']/kaggle_employ_share['total']*100,0)
kaggle_employ_share = kaggle_employ_share[['Gender','Age','Unemployed_share']]
ages = ['18-24', '25-29', '30-39', '40-49', '50-59','60+']

# unemployed rate in gender
fig = go.Figure()

# Man unemployed rate in age groups
man_unemployed = kaggle_employ_share.loc[kaggle_employ_share["Gender"] == 'Man']

# Add a line for man participants
fig.add_trace(go.Scatter(x=man_unemployed.Age, y=man_unemployed.Unemployed_share,
                    mode='lines',
                    name='Man',
                    line = dict(color = '#9fc5e8'
                                  ,width=5))
             )

# Add markers at the line for man participants
fig.add_trace(go.Scatter(
        x=[man_unemployed.Age.to_list()[1],
           man_unemployed.Age.to_list()[2],
           man_unemployed.Age.to_list()[3]],
        y=[man_unemployed.Unemployed_share.to_list()[1],
          man_unemployed.Unemployed_share.to_list()[2],
          man_unemployed.Unemployed_share.to_list()[3]],
        mode='markers',
        name = 'Man',
    showlegend=False,
        marker=dict(color='#9fc5e8', size=15)
    ))

# Add a line for woman participants
woman_unemployed = kaggle_employ_share.loc[kaggle_employ_share["Gender"] == 'Woman']
fig.add_trace(go.Scatter(x=woman_unemployed.Age, y=woman_unemployed.Unemployed_share,
                    mode='lines',
                    name='Woman',
                    line = dict(color = '#ea9999'
                                  ,width=5))
             )

# Add markers at the line for woman participants
fig.add_trace(go.Scatter(
        x=[woman_unemployed.Age.to_list()[1],
           woman_unemployed.Age.to_list()[2],
           woman_unemployed.Age.to_list()[3]],
        y=[woman_unemployed.Unemployed_share.to_list()[1],
          woman_unemployed.Unemployed_share.to_list()[2],
          woman_unemployed.Unemployed_share.to_list()[3]],
        mode='markers',
        name = 'Woman',
        showlegend=False,
        marker=dict(color='#ea9999', size=15)
    ))


# layout for x-axis, y-axis, chart size, chart background color
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=100,
        t=110,
    ),
    #showlegend=False,
    plot_bgcolor='white'
)


annotations = []

# Labels for age 25-29, 30-39, 40-49
annotations.append(dict(xref='paper', x=0.35, y=man_unemployed.Unemployed_share.to_list()[1],
                                  xanchor='right', yanchor='middle',
                                  text='Man' + ' {}%'.format(man_unemployed.Unemployed_share.to_list()[1]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))

annotations.append(dict(xref='paper', x=0.35, y=woman_unemployed.Unemployed_share.to_list()[1],
                                  xanchor='right', yanchor='middle',
                                  text='Woman' + ' {}%'.format(woman_unemployed.Unemployed_share.to_list()[1]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='% of unemployed survey participants in each age group'\
                                      '<br><span style="font-size:12px"><i>The unemployed rate in'\
                                      ' woman participants surpassed man since age 25-29</span></i>',
                              font=dict(family='Arial',
                                        size=16,
                                        color='rgb(37,37,37)'),
                              showarrow=False))



fig.update_layout(annotations=annotations)

fig.show()

Finding 2:

Higher shares of participants are students in woman than in man.

In [ ]:
# % of Studens in employed survey participants in gender
kaggle_student = kaggle.loc[kaggle['Employed'] == 'Employed'][['Gender','Age','Title','Count']]

# add a new column 'Student' transformed from 'Title' (Job Titles)
def student(row):
    if row == 'NaN':
        return 'Not answered'
    elif row == 'Student':
        return 'Student'
    else:
        return 'Not Student'

kaggle_student['Student'] = kaggle_student['Title'].apply(lambda row: student(row))

kaggle_student = kaggle_student.groupby(['Gender','Age','Student'])['Count'].sum().reset_index()
kaggle_student_total = kaggle_student.groupby(['Gender','Age'])['Count'].sum().reset_index()
kaggle_student_total.columns = kaggle_student_total.columns.str.replace('Count', 'total')
kaggle_student_share = pd.merge(
    kaggle_student,
    kaggle_student_total,
    how="inner",
    on=['Gender','Age'],
    sort=True,
    copy=True,
    indicator=False,
    validate=None
)

kaggle_student_share['student_share'] = round(kaggle_student_share['Count']/kaggle_student_share['total']*100,0)
kaggle_student_share = kaggle_student_share.loc[kaggle_student_share['Student'] == 'Student']
kaggle_student_share = kaggle_student_share[['Gender','Age','student_share']]
ages = ['18-24', '25-29', '30-39', '40-49', '50-59','60+']
kaggle_student_share['student_share'] = kaggle_student_share['student_share'].fillna(0)

# student rate in gender

fig = go.Figure()

# Man student rate in age groups
man_student = kaggle_student_share.loc[kaggle_student_share["Gender"] == 'Man']

# Add a line for man participants
fig.add_trace(go.Scatter(x=man_student.Age, y=man_student.student_share,
                    mode='lines',
                    name='Man',
                    line = dict(color = '#9fc5e8'
                                  ,width=5))
             )

# Add markers at the line for man participants
fig.add_trace(go.Scatter(
        x=[man_student.Age.to_list()[1]],
        y=[man_student.student_share.to_list()[1]],
        mode='markers',
        name = 'Man',
    showlegend=False,
        marker=dict(color='#9fc5e8', size=15)
           ))
    

# Add a line for woman participants
woman_student = kaggle_student_share.loc[kaggle_student_share["Gender"] == 'Woman']
fig.add_trace(go.Scatter(x=woman_student.Age, y=woman_student.student_share,
                    mode='lines',
                    name='Woman',
                    line = dict(color = '#ea9999'
                                  ,width=5))
             )

# Add markers at the line for woman participants
fig.add_trace(go.Scatter(
        x=[woman_student.Age.to_list()[1]],
        y=[woman_student.student_share.to_list()[1]],
        mode='markers',
        name = 'Woman',
        showlegend=False,
        marker=dict(color='#ea9999', size=15)  
    ))


# layout for x-axis, y-axis, chart size, chart background color
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=100,
        t=110,
    ),
    #showlegend=False,
    plot_bgcolor='white'
)


annotations = []

# Labels for age 25-29, 30-39, 40-49
annotations.append(dict(xref='paper', x=0.35, y=man_student.student_share.to_list()[1],
                                  xanchor='right', yanchor='middle',
                                  text='Man' + ' {}%'.format(man_student.student_share.to_list()[1]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))

annotations.append(dict(xref='paper', x=0.35, y=woman_student.student_share.to_list()[1],
                                  xanchor='right', yanchor='middle',
                                  text='Woman' + ' {}%'.format(woman_student.student_share.to_list()[1]),
                                  font=dict(family='Arial',
                                            size=16),
                                  showarrow=False))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='% of survey participants that are students in each age group'\
                                      '<br><span style="font-size:12px"><i>Higher shares of participants '\
                                      'are students in woman than man </span></i>',
                              font=dict(family='Arial',
                                        size=16,
                                        color='rgb(37,37,37)'),
                              showarrow=False))



fig.update_layout(annotations=annotations)

fig.show()

Finding 3:

Women are more likely to pursue higher academic levels than men.

In [ ]:
# % of employed survey participants having different highest education level
kaggle_edu = kaggle.loc[kaggle['Employed'] == 'Employed'][['Gender','Age','HighestEdu','Count']]
kaggle_edu = kaggle_edu.loc[kaggle_edu['HighestEdu'] != 'I prefer not to answer']

# grouping education levels
kaggle_edu.HighestEdu = kaggle_edu.HighestEdu.replace("Some college/university study without earning a bachelor’s degree","Bachelor or below")
kaggle_edu.HighestEdu = kaggle_edu.HighestEdu.replace("Bachelor’s degree","Bachelor or below")
kaggle_edu.HighestEdu = kaggle_edu.HighestEdu.replace("Professional degree","Bachelor or below")
kaggle_edu.HighestEdu = kaggle_edu.HighestEdu.replace("No formal education past high school","Bachelor or below")

# add a new column 'Bachelor_master' transformed from 'HighestEdu'
def bachelor_master(row):
    if row == 'Master’s degree':
        return 'Master or above'
    elif row == 'Doctoral degree':
        return 'Master or above'
    else:
        return 'Bachelor or below'

kaggle_edu['Bachelor_master'] = kaggle_edu['HighestEdu'].apply(lambda row: bachelor_master(row))

kaggle_edu = kaggle_edu.groupby(['Gender','Age','Bachelor_master'])['Count'].sum().reset_index()
kaggle_edu_total = kaggle_edu.groupby(['Gender','Age'])['Count'].sum().reset_index()
kaggle_edu_total.columns = kaggle_edu_total.columns.str.replace('Count', 'total')
kaggle_edu_share = pd.merge(
    kaggle_edu,
    kaggle_edu_total,
    how="inner",
    on=['Gender','Age'],
    sort=True,
    copy=True,
    indicator=False,
    validate=None
)


kaggle_edu_share['Edu_share'] = round(kaggle_edu_share['Count']/kaggle_edu_share['total']*100,0)
#kaggle_edu_share = kaggle_edu_share.loc[kaggle_student_share['Student'] == 'Student']
kaggle_edu_share = kaggle_edu_share[['Gender','Age','Bachelor_master','Edu_share']]

# plot the share of participants who have master degree above
# Add data
ages = ['18-24', '25-29', '30-39', '40-49', '50-59','60+']
man_edu = kaggle_edu_share.loc[kaggle_edu_share['Gender']== "Man"]
woman_edu = kaggle_edu_share.loc[kaggle_edu_share['Gender']== "Woman"]


man_bachelor = man_edu.loc[man_edu['Bachelor_master']== "Bachelor or below"].Edu_share
woman_bachelor = woman_edu.loc[woman_edu['Bachelor_master']== "Bachelor or below"].Edu_share
man_master = man_edu.loc[man_edu['Bachelor_master']== "Master or above"].Edu_share
woman_master = woman_edu.loc[woman_edu['Bachelor_master']== "Master or above"].Edu_share

# dash options include 'dash', 'dot', and 'dashdot'
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=ages, y=man_master, name='Man',
                         line=dict(color='#9fc5e8', width=4)))
fig.add_trace(go.Scatter(x=ages, y=woman_master, name='Woman',
                         line = dict(color='#ea9999', width=4)))



# layout for x-axis, y-axis, chart size, chart background color
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=30,
        t=110,
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.01,
    xanchor="right",
    x=1
    ),
    #showlegend=False,
    plot_bgcolor='white'
)


annotations = []



# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='% of participants having master or doctoral degree in the next 2 years.'\
                                      '<br><span style="font-size:12px"><i>Women are more likely to '\
                                      'pursue higher academic levels than men.</span></i>',
                              font=dict(family='Arial',
                                        size=16,
                                        color='rgb(37,37,37)'),
                              showarrow=False))



fig.update_layout(annotations=annotations)
fig.show()


Finding 4:

Women with master or even doctor degree can win an equal amount of median yearly salary with men having bachelor degree.

In [ ]:
# Median yearly compensation in gender
kaggle_compensation = kaggle.loc[kaggle['Employed'] == 'Employed'][['Gender','Age','Title','HighestEdu','Compensation','Count']]

# add a new column 'Bachelor_master' transformed from 'HighestEdu'
def bachelor_master(row):
    if row == 'Master’s degree':
        return 'Master or above'
    elif row == 'Doctoral degree':
        return 'Master or above'
    else:
        return 'Bachelor or below'

kaggle_compensation['Bachelor_master'] = kaggle_compensation['HighestEdu'].apply(lambda row: bachelor_master(row))

# Removing those who didn't disclose compensation
answers_before = kaggle_compensation.shape[0]
kaggle_compensation.dropna(subset=['Compensation'], inplace=True)
answers_after = kaggle_compensation.shape[0]
#print(f'After removing respondents who did not disclose compensation there were left {answers_after} answers.')

# Transform range of salaries into numerical value
# We are summing up the lowest and highest value for each category, and then dividing by 2.
# Some regex needed to clean the text
kaggle_compensation.Compensation = kaggle_compensation.Compensation.replace('$0-999', '0-999')
kaggle_compensation.Compensation = kaggle_compensation.Compensation.replace('> $500,000', '500,000-500,000')
compensation = kaggle_compensation.Compensation.str.replace(r'(?:(?!\d|\-).)*', '').str.split('-')
kaggle_compensation['Compensation'] = compensation.apply(lambda x: round((float(x[0]) + float(x[1]))/ 2),-3)

kaggle_compensation = kaggle_compensation.groupby(['Gender','Age','Bachelor_master'])['Compensation'].median().reset_index()

# Plot median yearly compensation in man-bachelor or below, woman-bachelor or below, man-master or above and woman-master or above
ages = ['18-24', '25-29', '30-39', '40-49', '50-59','60+']
man_compensation = kaggle_compensation.loc[kaggle_compensation['Gender']== "Man"]
woman_compensation = kaggle_compensation.loc[kaggle_compensation['Gender']== "Woman"]


man_bachelor = man_compensation.loc[man_compensation['Bachelor_master']== "Bachelor or below"].Compensation
man_master = man_compensation.loc[man_compensation['Bachelor_master']== "Master or above"].Compensation
woman_bachelor = woman_compensation.loc[woman_compensation['Bachelor_master']== "Bachelor or below"].Compensation
woman_master = woman_compensation.loc[woman_compensation['Bachelor_master']== "Master or above"].Compensation

"""
man_doctor = man_edu.loc[man_edu['Bachelor_master']== "Doctoral degree"].Edu_share
woman_doctor = woman_edu.loc[woman_edu['Bachelor_master']== "Doctoral degree"].Edu_share

fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=True,
                    shared_yaxes=True,
                    vertical_spacing=0.1)
"""
# dash options include 'dash', 'dot', and 'dashdot'
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=ages, y=man_bachelor, name='Bachelor or below[Man]',
                         line=dict(color='#9fc5e8', width=4)))

fig.add_trace(go.Scatter(x=ages, y=man_master, name='Master or above[Man]',
                         line=dict(color='#D9E2E9', width=2,dash ='dash')))


fig.add_trace(go.Scatter(x=ages, y=woman_bachelor, name = 'Bachelor or below[Woman]',
                         line=dict(color='#D9E2E9', width=2)))

fig.add_trace(go.Scatter(x=ages, y=woman_master, name='Master or above[Woman]',
                         line=dict(color='#ea9999', width=4,dash='dash')))



# layout for x-axis, y-axis, chart size, chart background color
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=30,
        t=110,
    ),
    legend=dict(
    orientation="h",
    yanchor="top",
    y=1,
    xanchor="left",
    x=0.01
    ),
    showlegend=True,
    plot_bgcolor='white'
)


annotations = []



# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Median yearly compensation in gender with different highest education levels'\
                                      '<br><span style="font-size:12px"><i>Women with master or even doctor degree '\
                                      'can win an equal amount of median yearly salary with men having bachelor degree.</span></i>',
                              font=dict(family='Arial',
                                        size=16,
                                        color='rgb(37,37,37)'),
                              showarrow=False))



fig.update_layout(annotations=annotations)
fig.show()
